<a href="https://colab.research.google.com/github/bdemirc1/chemBERT/blob/main/ChemBERT_10K.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
# Nvidia Apex for dealing with 16-bit training-less memory but mixed precision training
# Apex alleviates these problems.
!git clone https://github.com/NVIDIA/apex
%cd apex
!pwd


Cloning into 'apex'...
remote: Enumerating objects: 11396, done.
remote: Counting objects: 100% (3533/3533), done.
remote: Compressing objects: 100% (423/423), done.
remote: Total 11396 (delta 3251), reused 3184 (delta 3108), pack-reused 7863
Receiving objects: 100% (11396/11396), 15.39 MiB | 26.75 MiB/s, done.
Resolving deltas: 100% (8010/8010), done.
/content/apex
/content/apex


In [25]:
!pip install -v --disable-pip-version-check --no-cache-dir --global-option="bdist_wheel" --global-option="--cpp_ext" --global-option="--cuda_ext" ./

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
DEPRECATION: --build-option and --global-option are deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use --config-settings. Discussion can be found at https://github.com/pypa/pip/issues/11859
Processing /content/apex
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.9 MB/s eta 0:00:00
    Getting requirements to build wheel: started
    Running command Getting requirements to build wheel
    running egg_info
    writing setuptools.egg-info/PKG-INFO
    writing dependency_links to setuptools.egg-info/dependency_links.txt
    writing entry points to setuptools.egg-info/entry_points.txt
    writing requirements to setuptools.egg-info/requires.txt
    writing top-level names to setuptools.egg-info/top_level.txt
    readi

In [26]:
# Required for simple transformer classification model - trained using f16 - true: 16 bit
from apex import amp

In [27]:
!pip install transformers
!pip install simpletransformers

In [28]:
!nvidia-smi

Tue Nov  7 23:55:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P0    30W /  70W |   2859MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
import os

import numpy as np
import pandas as pd


In [3]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")  # GPU available
else:
    device = torch.device("cpu")  # No GPU, fall back to CPU
device

device(type='cuda')

In [11]:
!pip install transformers
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 24.8 MB/s eta

In [29]:
!pip install wandb

In [12]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [5]:
df = pd.read_csv("10k_dft_density_data.csv")
df.dtypes
df.info

<bound method DataFrame.info of                                                   smiles   density
0                           CC(C)OC(=O)c1ccc(cc1)N(=O)=O  1.367633
1                 CCc1ccc(cc1)C=C(COc1ccccc1C=NO)C(=O)OC  1.292618
2                              CC(C)(C)c1ccc(O)c(C=NO)c1  1.227993
3                      Cc1ccccc1NN=C(C)c1cccc(c1)N(=O)=O  1.316755
4      CN(C)c1ccc(cc1)C(=C(C#N)C#N)C(c1ccc(cc1)N(=O)=...  1.406207
...                                                  ...       ...
10201         CON1C(=C(OC(C)=O)C(=C1C=Cc1ccccc1)C(C)=O)C  1.299822
10202                        CON=C(C)C(=O)C=CC=Cc1ccccc1  1.220360
10203                          CON=C(C)C(=O)C=CC1=CC=CN1  1.261450
10204      CN(c1ccccn1)c1c(cc(cc1N(=O)=O)N(=O)=O)N(=O)=O  1.579706
10205  Cc1c(nc2CC(C)(C)CC(=O)c2c1c1ccc(cc1)N(=O)=O)c1...  1.333787

[10206 rows x 2 columns]>

In [6]:
new_column_names = {
    "smiles": "text",
    "density": "labels",
}
df.rename(columns=new_column_names, inplace=True)
df

,text,labels
0,CC(C)OC(=O)c1ccc(cc1)N(=O)=O,1.367633
1,CCc1ccc(cc1)C=C(COc1ccccc1C=NO)C(=O)OC,1.292618
2,CC(C)(C)c1ccc(O)c(C=NO)c1,1.227993
3,Cc1ccccc1NN=C(C)c1cccc(c1)N(=O)=O,1.316755
4,CN(C)c1ccc(cc1)C(=C(C#N)C#N)C(c1ccc(cc1)N(=O)=...,1.406207
...,...,...
10201,CON1C(=C(OC(C)=O)C(=C1C=Cc1ccccc1)C(C)=O)C,1.299822
10202,CON=C(C)C(=O)C=CC=Cc1ccccc1,1.220360
10203,CON=C(C)C(=O)C=CC1=CC=CN1,1.261450
10204,CN(c1ccccn1)c1c(cc(cc1N(=O)=O)N(=O)=O)N(=O)=O,1.579706


In [7]:
from sklearn.model_selection import train_test_split

train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)
validation_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

In [8]:
train_df

,text,labels
7539,CN1OC(C)(C)CC1(C#N)C1COC2(CCCCC2)O1,1.252277
8806,CC(=O)C1=CN(C=C(C1c1ccc(cc1)N(=O)=O)C(C)=O)c1c...,1.365263
9560,CC(C)C1=NC2=C(O1)c1ccccc1C(=O)C2=NOC(C)=O,1.391895
6425,CC1C2C(CON2C(C)(C)C)CN1CC=C,1.139091
8511,CN(C)CCN(Cc1c2ccccc2cc2ccccc12)c1ccc(cn1)N(=O)=O,1.319714
...,...,...
5734,CC(=O)OCC1OC(Oc2ccc(cc2N(=O)=O)N(=O)=O)C(OC(C)...,1.531870
5191,CC(C)(C)C(CN(=O)=O)C(=O)c1ccc2OCOc2c1,1.371337
5390,CC1(C)CC(=O)C2=C(C1)OC(=NO)C2c1cccnc1,1.344472
860,CON1C(=O)c2ccccc2C1=CC(=O)NC(C)C,1.312522


In [18]:
test_df.columns

Index(['text', 'labels'], dtype='object')

In [34]:
model_args = ClassificationArgs()
model_args.num_train_epochs = 1
model_args.regression = True
model_args.evaluate_each_epoch = True
model_args.evaluate_during_training_verbose = True
model_args.no_save = True
model_args.num_train_epochs = 5
model_args.wandb_project = 'chemBert10k-density'

In [35]:
!wandb login

wandb: Currently logged in as: bdemirc1 (bingedu). Use `wandb login --relogin` to force relogin


In [36]:
# Create a ClassificationModel
model = ClassificationModel(
    "roberta",
    "seyonec/ChemBERTA_PubChem1M_shard00_155k",
    num_labels=1,
    args=model_args
)

# Train the model
model.train_model(train_df)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at seyonec/ChemBERTA_PubChem1M_shard00_155k and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/7144 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

wandb: Currently logged in as: bdemirc1 (bingedu). Use `wandb login --relogin` to force relogin


Running Epoch 0 of 5:   0%|          | 0/893 [00:00<?, ?it/s]

...

Running Epoch 1 of 5:   0%|          | 0/893 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/893 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/893 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/893 [00:00<?, ?it/s]

(4465, 0.016998380361008068)

In [37]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(validation_df)

  0%|          | 0/1531 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/192 [00:00<?, ?it/s]

In [38]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Convert text input data to list of strings
text_inputs = test_df['text'].tolist()

# Predict target values
predictions, model_outputs = model.predict(text_inputs)

# Calculate MAE and MSE
mae = mean_absolute_error(test_df['labels'], predictions)
mse = mean_squared_error(test_df['labels'], predictions)
rmse = np.sqrt(mse)

# Print the results
print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)


  0%|          | 0/1531 [00:00<?, ?it/s]

  0%|          | 0/192 [00:00<?, ?it/s]

Mean Absolute Error: 0.07377328864035591
Mean Squared Error: 0.005815321432123875
Root Mean Squared Error: 0.07625825484577965
